# Замеры скорости
В этом файле производятся замеры скорости обучения и инференса иерархической и плоской моделей

In [1]:
import pandas as pd

df = pd.read_csv("data/train_40k.csv").dropna(how='any')

# Define a list with sentences (1k - 100k sentences)
train_sentences = df['Text'].tolist()
labels = df[['Cat1', 'Cat2', 'Cat3']].to_numpy()
labels

array([['grocery gourmet food', 'meat poultry', 'jerky'],
       ['toys games', 'games', 'unknown'],
       ['toys games', 'games', 'unknown'],
       ...,
       ['beauty', 'tools accessories', 'mirrors'],
       ['beauty', 'skin care', 'body'],
       ['beauty', 'fragrance', 'women s']], dtype=object)

~~Балансировка классов предлагает молиться~~. Согласно отчёту, в датасете 0.7% дубликатов, все одного класса

### Создание иерархии

In [2]:
def create_hierarchy(df):
    hierarchy = {}

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        current_level = hierarchy  # Start from the root of the hierarchy

        # Build the path based on the values in the current row
        for col in df.columns:
            label = row[col]
            if pd.notna(label):  # Check if the label is not NaN
                # If the label doesn't exist in the current level, add it
                if label not in current_level:
                    current_level[label] = {}
                # Move down to the next level in the hierarchy
                current_level = current_level[label]

    return hierarchy

hierarchy = {'root': create_hierarchy(df[['Cat1', 'Cat2', 'Cat3']])}

 Обучение TF-IDF и иерархического классификатора
Помимо TF-IDF на unlabeled текстах был дообучен DistilRoberta, но точность на эмбеддингах Bert и на TF-IDF не отличается

In [3]:
pretrain_texts = pd.read_csv("data/unlabeled_150k.csv").dropna(how='any')['Text'].tolist()
tfidf_text = pretrain_texts + train_sentences

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
import time

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(tfidf_text)
embeddings = tfidf_vectorizer.transform(train_sentences)

In [5]:
from model_serving.app.services.lib import TopDownClassifier
from sklearn.tree import DecisionTreeClassifier

base_classifier = DecisionTreeClassifier
top_down_clf = TopDownClassifier(base_classifier, hierarchy)
topdown_start_train_time = time.time()
top_down_clf.fit(embeddings, labels)
topdown_end_train_time = time.time()

In [6]:
df = pd.read_csv("data/val_10k.csv").dropna(how='any')

test_sentences = df['Text'].tolist()
test_labels = df[['Cat1', 'Cat2', 'Cat3']].to_numpy()

In [7]:
test_embeddings = tfidf_vectorizer.transform(test_sentences)

In [8]:
topdown_start_predict_time = time.time()
top_down_clf.predict(test_embeddings)
topdown_end_predict_time = time.time()

# Обучение плоского классификатора
**В силу того, что плоская логрегрессия обучалась два часа, съела 20 Гб RAM и не показала никакого результата, дальше обучен DecisionTreeClassifier**

In [9]:
import pandas as pd

df = pd.read_csv("data/train_40k.csv").dropna(how='any')
train_sentences = df['Text'].tolist()
train_labels = df[['Cat1', 'Cat2', 'Cat3']].to_numpy()
train_labels = train_labels[:,0] + "_" + train_labels[:,1] + "_" + train_labels[:,2]

In [10]:
df = pd.read_csv("data/val_10k.csv").dropna(how='any')
test_sentences = df['Text'].tolist()
test_labels = df[['Cat1', 'Cat2', 'Cat3']].to_numpy()
test_labels = test_labels[:,0] + "_" + test_labels[:,1] + "_" + test_labels[:,2]

In [11]:
from sklearn.tree import DecisionTreeClassifier

train_embeddings = tfidf_vectorizer.transform(train_sentences)
regressor = DecisionTreeClassifier(random_state=42)
flat_start_train_time = time.time()
regressor.fit(train_embeddings, train_labels)
flat_end_train_time = time.time()

In [12]:
test_embeddings = tfidf_vectorizer.transform(test_sentences)
flat_start_predict_time = time.time()
regressor.predict(test_embeddings)
flat_end_predict_time = time.time()

# Время обучения и инференса

In [16]:
print(f'Тренировка плоского классификатора: {flat_end_train_time - flat_start_train_time:.4f} c')
print(f'Инференс плоского классификатора: {flat_end_predict_time - flat_start_predict_time:.4f} c')
print(f'Тренировка иерархического классификатора: {topdown_end_train_time - topdown_start_train_time:.4f} c')
print(f'Инференс иерархического классификатора: {topdown_end_predict_time - topdown_start_predict_time:.4f} c')

Тренировка плоского классификатора: 197.8788 c
Инференс плоского классификатора: 0.0240 c
Тренировка иерархического классификатора: 59.1053 c
Инференс иерархического классификатора: 5.3128 c


Иерархический классификатор обучается значительно быстрее, но в инференсе он значительно медленнее из-за накладных расходов вычисления несколькими базовыми классификаторами.
Скорее всего, иерархический классификатор быстро обучается из-за малого времени схождения алгоритма для малого количества классов, базовые классификаторы иерархии обучают не более чем десятки классов, когда плоский должен выучить за тысячу, и сходится он значительно медленнее

In [22]:
from sklearn.pipeline import Pipeline
import pickle

p = Pipeline([('tfidf_vectorizer', tfidf_vectorizer), ('classifier', top_down_clf)])

with open("model_serving/app/models/model.pkl", "wb") as f:
    pickle.dump(p, f)

In [23]:
type(p.predict(['хуй']))

numpy.ndarray